**In this notebook use the idea of portfolio**
https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction/discussion/318055

# **if it is useful for you, please vote!**

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")

In [ ]:
prices["Date"]

In [ ]:
NDAYS = 50
lastdays = prices[prices["Date"]>=prices.Date.iat[-2000*NDAYS]].reset_index(drop=True)

In [ ]:
lastdays = pd.DataFrame(prices.groupby("SecuritiesCode").Target.mean())
def get_avg(_id_):
    return lastdays.loc[_id_]
prices["Avg"] = prices["SecuritiesCode"].apply(get_avg)

In [ ]:
prices.Date = pd.to_datetime(prices.Date)
prices['Date'] = prices['Date'].dt.strftime("%Y%m%d").astype(int)
X=prices[["Date","SecuritiesCode","Avg","High","Open","Close","Low","Volume"]]

y=prices[["Target"]]
codes = X.SecuritiesCode.unique()

In [ ]:
def fill_nans(prices):
    prices.ffill(inplace=True)
    prices.fillna(0,inplace=True)
    return prices

I get a problem with nan in cell with optuna, so try to fill nan in simple way

In [ ]:
X.isnull().sum()

In [ ]:
y.isnull().sum()

In [ ]:
%%time
X = fill_nans(X)

pd.options.display.float_format = '{:,.6g}'.format
X.describe()

In [ ]:
X.isnull().sum()

In [ ]:
y = fill_nans(y)
y.isnull().sum()

Now its okay and ready to fit model

In [ ]:
import optuna

def objectives(trial):
    # Setting the hyperparameter search range
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 300, 4000),
            'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
            'max_bin': trial.suggest_int('max_bin', 2, 100),
            'learning_rate': trial.suggest_uniform('learning_rate',0, 1),
    }

    model = LGBMRegressor(**params)
    model.fit(X,y)
    score = model.score(X,y)
    return score

In [ ]:
opt = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=0))
opt.optimize(objectives, n_trials=20)

# Obtaining optimal parameters
trial = opt.best_trial
params_best = dict(trial.params.items())
params_best['random_seed'] = 0
    
# Training/prediction with optimal parameters
model_o = LGBMRegressor(**params_best)#

In [ ]:
model_o.fit(X,y)

And submit with api

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
    sample_prediction["Avg"] = sample_prediction["SecuritiesCode"].apply(get_avg)
    df = sample_prediction[["Date","SecuritiesCode","Avg"]]
    df["High"] = prices["High"]
    df["Open"] = prices["Open"]
    df["Close"] = prices["Close"]
    df["Low"] = prices["Low"]
    df["Volume"] = prices["Volume"]
    df.Date = pd.to_datetime(df.Date)
    sample_prediction["Volume"] = df["Volume"]
    df['Date'] = df['Date'].dt.strftime("%Y%m%d").astype(int)
    sample_prediction["Prediction"] = model_o.predict(df)
    sample_prediction["rate"] = sample_prediction["Prediction"]/(np.log(sample_prediction["Volume"]+1))
    sample_prediction = sample_prediction.sort_values(by = "rate", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)